In [27]:
import ee
from IPython.display import clear_output
import numpy as np
import rasterio
from glob import glob
import time
import subprocess
import json
import pandas as pd
import sys
sys.path.append('../func')
from wrapper_rasterio import export_tiff
# from pathlib import Path
# from urllib.request import urlretrieve
# import zipfile

# ee.Authenticate()
ee.Initialize()

# S2 Download

Define several functions to be applied on EE

In [2]:
def maskS2clouds(image):
    """
    masking cloud for sentinel2 with the band "QA60".
    """
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).select("B.*").copyProperties(image, ["system:time_start"])

def calc_NDVI(image):
    NDVI = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

    return NDVI.copyProperties(image, ["system:time_start"])

def calc_ECARR(image):    

    ECARR = image.expression(
        '0.0008 * (10000 * B8A / (B3 * B5)) ** 1.2403', {
            'B3': image.select('B3'),
            'B5': image.select('B5'),
            'B8A': image.select('B8A')
        }).rename('ECARR')


#     # Divide does not work
#     ECARR = image.divide(10000).expression(
#         '0.0008 * (B8A / (B3 * B5)) ** 1.2403', {
#             'B3': image.select('B3'),
#             'B5': image.select('B5'),
#             'B8A': image.select('B8A')
#         }).rename('ECARR')
    
    return ECARR.copyProperties(image, ["system:time_start"])

Load an AOI for downloading  
Make sure the CRS is **EPSG:4326**

In [28]:
# with open('./data/aoi/Albany_sub.geojson','r') as f:
#     aoi_json = json.load(f)

with open('./data/aoi/Albany_150km_rec.geojson','r') as f:
    aoi_json = json.load(f)

# with open('./data/aoi/Esperance_150km.geojson','r') as f:
#     aoi_json = json.load(f)

coords = aoi_json['features'][0]['geometry']['coordinates'][0][0]
aoi = ee.Geometry.Polygon(coords)

Download an ECARR image and export to a bucket

In [21]:
epsg = 28350
resolution = 20
gs_bucket = 'takahata-dev'

In [13]:
img_collection = (ee.ImageCollection(f'COPERNICUS/S2_SR')
                  .select(['B3','B5','B8A','QA60'])
                  .filterBounds(aoi)
                  .filterDate(ee.Date('2019-12-01'), ee.Date('2019-12-31'))
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
#                   .filter(ee.Filter.lt('NODATA_PIXEL_PERCENTAGE',5))
                  .map(maskS2clouds)
                  .map(calc_ECARR))

info = img_collection.getInfo()
info_date = info['features'][0]['properties']['system:index']

img = ee.Image(img_collection.median()) # Take the median for a month

task = ee.batch.Export.image.toCloudStorage(image=img,  # an ee.Image object.
                                            region=aoi,  # an ee.Geometry object.
                                            description='Download S2 data',
                                            bucket=gs_bucket,   # Must be a root directory. E.g. "d1/d2" does not work
                                            fileNamePrefix=f'sentinel2/ECARR_Albany_{info_date}', # Target directory can be specified here
                                            crs=f'EPSG:{epsg}',
                                            scale=resolution, # meter per pixel
                                            maxPixels=1e+10
                                           )

task.start()
while True:
    clear_output(wait=True)
    status = task.status()
    print(status)
    if status['state'] == 'COMPLETED' or status['state'] == 'FAILED':
        break
    time.sleep(2)

{'state': 'COMPLETED', 'description': 'Download S2 data', 'creation_timestamp_ms': 1639018590397, 'update_timestamp_ms': 1639019119329, 'start_timestamp_ms': 1639018616079, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/takahata-dev/sentinel2/'], 'attempt': 1, 'id': 'PKVI3QIZ6ECV2DGL7TJ7KHPQ', 'name': 'projects/earthengine-legacy/operations/PKVI3QIZ6ECV2DGL7TJ7KHPQ'}


In [14]:
img_collection = (ee.ImageCollection(f'COPERNICUS/S2_SR')
                  .select(['B3','B5','B8A','QA60'])
                  .filterBounds(aoi)
                  .filterDate(ee.Date('2020-12-01'), ee.Date('2020-12-31'))
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
#                   .filter(ee.Filter.lt('NODATA_PIXEL_PERCENTAGE',5))
                  .map(maskS2clouds)
                  .map(calc_ECARR))

info = img_collection.getInfo()
info_date = info['features'][0]['properties']['system:index']

img = ee.Image(img_collection.median()) # Take the median for a month

task = ee.batch.Export.image.toCloudStorage(image=img,  # an ee.Image object.
                                            region=aoi,  # an ee.Geometry object.
                                            description='Download S2 data',
                                            bucket=gs_bucket,   # Must be a root directory. E.g. "d1/d2" does not work
                                            fileNamePrefix=f'sentinel2/ECARR_Albany_{info_date}', # Target directory can be specified here
                                            crs=f'EPSG:{epsg}',
                                            scale=resolution, # meter per pixel
                                            maxPixels=1e+10
                                           )

task.start()
while True:
    clear_output(wait=True)
    status = task.status()
    print(status)
    if status['state'] == 'COMPLETED' or status['state'] == 'FAILED':
        break
    time.sleep(2)

{'state': 'COMPLETED', 'description': 'Download S2 data', 'creation_timestamp_ms': 1639019120860, 'update_timestamp_ms': 1639021029567, 'start_timestamp_ms': 1639019133015, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/takahata-dev/sentinel2/'], 'attempt': 1, 'id': 'SGELGXZ6BWL5FFZP4JUULAUG', 'name': 'projects/earthengine-legacy/operations/SGELGXZ6BWL5FFZP4JUULAUG'}


In [27]:
# # Quick visualization (available only when the image is small)    
# visualization = {
#     "region": aoi,
#     "scale": 20,
#     "crs": 'EPSG:28350'
# }
# url = img.getDownloadURL(visualization)
# urlretrieve(url,'data/image/img_tmp.zip')

# with zipfile.ZipFile("data/image/img_tmp.zip","r") as zip_ref:
#     zip_ref.extractall(f"data/image/")

Download an NDVI image and export to a bucket

In [15]:
img_collection = (ee.ImageCollection(f'COPERNICUS/S2_SR')
                  .select(['B4','B8','QA60'])
                  .filterBounds(aoi)
                  .filterDate(ee.Date('2019-12-01'), ee.Date('2019-12-31'))
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
                  .map(maskS2clouds)
                  .map(calc_NDVI))

info = img_collection.getInfo()
info_date = info['features'][0]['properties']['system:index']


img = ee.Image(img_collection.median())
task = ee.batch.Export.image.toCloudStorage(image=img,
                                            region=aoi,
                                            description='Download S2 data',
                                            bucket=gs_bucket,
                                            fileNamePrefix=f'sentinel2/NDVI_Albany_{info_date}',
                                            crs=f'EPSG:{epsg}',
                                            scale=resolution, # meter per pixel
                                            maxPixels=1e+10)

task.start()
while True:
    clear_output(wait=True)
    status = task.status()
    print(status)
    if status['state'] == 'COMPLETED' or status['state'] == 'FAILED':
        break
    time.sleep(2)

{'state': 'COMPLETED', 'description': 'Download S2 data', 'creation_timestamp_ms': 1639021032452, 'update_timestamp_ms': 1639021631926, 'start_timestamp_ms': 1639021050597, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/takahata-dev/sentinel2/'], 'attempt': 1, 'id': '42SIQVXP646WCCZT6RA2256G', 'name': 'projects/earthengine-legacy/operations/42SIQVXP646WCCZT6RA2256G'}


In [22]:
img_collection = (ee.ImageCollection(f'COPERNICUS/S2_SR')
                  .select(['B4','B8','QA60'])
                  .filterBounds(aoi)
                  .filterDate(ee.Date('2020-12-01'), ee.Date('2020-12-31'))
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
                  .map(maskS2clouds)
                  .map(calc_NDVI))

info = img_collection.getInfo()
info_date = info['features'][0]['properties']['system:index']


img = ee.Image(img_collection.median())
task = ee.batch.Export.image.toCloudStorage(image=img,
                                            region=aoi,
                                            description='Download S2 data',
                                            bucket=gs_bucket,
                                            fileNamePrefix=f'sentinel2/NDVI_Albany_{info_date}',
                                            crs=f'EPSG:{epsg}',
                                            scale=resolution, # meter per pixel
                                            maxPixels=1e+10)

task.start()
while True:
    clear_output(wait=True)
    status = task.status()
    print(status)
    if status['state'] == 'COMPLETED' or status['state'] == 'FAILED':
        break
    time.sleep(2)

{'state': 'COMPLETED', 'description': 'Download S2 data', 'creation_timestamp_ms': 1639022809141, 'update_timestamp_ms': 1639023193969, 'start_timestamp_ms': 1639022829717, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/takahata-dev/sentinel2/'], 'attempt': 1, 'id': '2A3WS6SUZ7W6EIBYYHROWUXU', 'name': 'projects/earthengine-legacy/operations/2A3WS6SUZ7W6EIBYYHROWUXU'}


After the export is completed, copy the files from the bucket to your local/VM machine

In [16]:
# Copy the downloaded data to the local/VM machine
loc = 'data/image/'
cmd = f'gsutil -m cp gs://takahata-dev/sentinel2/*.tif {loc}'
subprocess.call(cmd.split())

Copying gs://takahata-dev/sentinel2/ECARR_20191202T015619_20191202T020454_T50HNG.tif...
Copying gs://takahata-dev/sentinel2/ECARR_20201202T021349_20201202T022052_T50HMG.tif...
Copying gs://takahata-dev/sentinel2/ECARR_Esperance_20191202T015619_20191202T020454_T50HQJ.tif...
Copying gs://takahata-dev/sentinel2/ECARR_Esperance_20201201T015621_20201201T015638_T50HQJ.tif...
\ [4 files][  3.4 GiB/  3.4 GiB]   33.6 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://takahata-dev/sentinel2/NDVI_20191202T015619_20191202T020454_T50HNG.tif...
Copying gs://takahata-dev/sentinel2/NDVI_20201202T021349_20201202T022052_T50HMG.tif...
Copying gs://takahata-dev/sentinel2/NDVI_Esperance_20191202T015619_20191202T020454_T50HQJ.tif...
/ [7 files][  4.7 GiB/  

0

# Pre-processing for labels from the government data

In the government data, each pixel only has a unique ID where we need to refer to the definition table to get forest type, owner, etc.  
For our purpose, we convert it to a map where each pixel has a forest code so that we are able to know the forest type immediately.

In [6]:
label_gov = rasterio.open('data/aus_for20_indigenous/ind_for20_Albany_28350.tif').read(1)
value_types = np.sort(np.unique(label_gov))
value_types = value_types[(value_types > 0) & (value_types < 1642)]
lookup = pd.read_csv('data/aus_for20_indigenous/def_table.csv')

label_gov_forcode = np.zeros(label_gov.shape, dtype=np.uint16)
for v in value_types:
    code = lookup.loc[lookup.Value == v, 'FOR_CODE'].values[0]
    label_gov_forcode[label_gov == v] = code

export_tiff(label_gov_forcode,
            target='data/label/ind_for20_Albany_28350_forcode.tif',
            reference='data/aus_for20_indigenous/ind_for20_Albany_28350.tif',
            nodata=0)

# Pre-processing for S2

The image files and the labels need to have exactly the same width and height.  
If not, they need to be adjusted.

In [19]:
src = rasterio.open('data/label/ind_for20_Albany_28350_forcode.tif')
ulx, lry, lrx, uly = src.bounds

loc_source = 'data/image'
images = glob(f'{loc_source}/*Albany*.tif')
for image in images:
    product_name = image[len(loc_source)+1:-4]
    loc_input = f'{loc_source}/{product_name}.tif'
    loc_output = f'{loc_source}/{product_name}_tmp.tif'
    cmd = f'gdal_translate -projwin {ulx} {uly} {lrx} {lry} {loc_input} {loc_output}'
    subprocess.call(cmd.split())
    cmd = f'rm {loc_input}'
    subprocess.call(cmd.split())
    cmd = f'mv {loc_output} {loc_input}'
    subprocess.call(cmd.split())